In [72]:
import h5py
import pandas as pd
import numpy as np
import json, time
from sklearn.metrics import mean_absolute_error as MAE

In [73]:
f = open('./epinions_subset/user_mapping.json')
user_mapping = json.loads(f.read())
f.close()
f = open('./epinions_subset/item_mapping.json')
item_mapping = json.loads(f.read())
f.close()

In [74]:
def five_fold_split(matrix):
    split_matrices = []
    valid_indices = matrix.nonzero()
    test_set_size = int(0.2 * len(valid_indices[0]))
    for i in range(5):
        start = int(i * 0.2 * len(valid_indices[0]))
        end = int((i+1) * 0.2 * len(valid_indices[0]))
        test_indices = (valid_indices[0][start:end], valid_indices[1][start:end])
        
        if start > 0 and end < len(valid_indices[0]):
            train_indices = (np.append(valid_indices[0][0:start], valid_indices[0][end:len(valid_indices[0])]),
                                       np.append(valid_indices[1][0:start], valid_indices[1][end:len(valid_indices[0])]) )
        elif start > 0:
            train_indices = (valid_indices[0][0:start], valid_indices[1][0:start])
        elif end < len(valid_indices[0]):
            train_indices = (valid_indices[0][end:len(valid_indices[0])], valid_indices[1][end:len(valid_indices[0])])
        
        train_matrix = np.zeros(matrix.shape)
        train_matrix[train_indices] = matrix[train_indices]
        test_matrix = np.zeros(matrix.shape)
        test_matrix[test_indices] = matrix[test_indices]
        split_matrices.append((train_matrix, test_matrix))
    count = 0
    for tup in split_matrices:
        print (len(valid_indices[0]), len(tup[0].nonzero()[0]) + len(tup[1].nonzero()[0]))
    return split_matrices

In [75]:
def train_item_based(user_by_item):
    print (user_by_item.shape)
    mask = np.ones(user_by_item.shape)
    mask[user_by_item.nonzero()] = 0
    masked_matrix = np.ma.masked_array(user_by_item, mask=mask)
    return np.ma.corrcoef(masked_matrix, rowvar=False).data

In [76]:
def train_user_based(user_by_item):
    item_by_user = user_by_item.T
    print (item_by_user.shape)
    mask = np.ones(item_by_user.shape)
    mask[item_by_user.nonzero()] = 0
    masked_matrix = np.ma.masked_array(item_by_user, mask=mask)
    return np.ma.corrcoef(masked_matrix, rowvar=False).data

In [77]:
user_item_matrix = np.loadtxt('./epinions_subset/user_item_matrix.txt')
split_matrices = five_fold_split(user_item_matrix)

68640 68640
68640 68640
68640 68640
68640 68640
68640 68640


In [7]:
for i in range(5):
    print (i)
    item_similarity = train_item_based(split_matrices[i][0])
    np.savetxt('./epinions_subset/item_similarity_%d.txt'%i, item_similarity)

0
(3000, 1000)
1
(3000, 1000)
2
(3000, 1000)
3
(3000, 1000)
4
(3000, 1000)


In [79]:
def nmae_item_item(test_matrix, train_matrix, similarity_matrix):
    indices = test_matrix.nonzero()
    y_true = []
    y_pred = []
    for i,j in zip(indices[0], indices[1]):
        y_true.append(test_matrix[i][j])
        item_sum = np.sum(similarity_matrix[j]) - 1
        if item_sum == 0:
            predicted_rating = np.mean(train_matrix[:,j][train_matrix[:,j].nonzero()])
        else:
            predicted_rating = np.mean(train_matrix[:,j][train_matrix[:,j].nonzero()]) + np.dot(similarity_matrix[j].flatten(), train_matrix[i].flatten())/item_sum
        y_pred.append(np.round(predicted_rating))
    return MAE(y_true, y_pred)

In [80]:
##prediction item item
count = 0
for i in range(5):
    similarity = np.loadtxt('./epinions_subset/item_similarity_%d.txt'%i)
    mae = nmae_item_item(split_matrices[i][1], split_matrices[i][0], similarity)
    print (mae)
    count += mae
print (count/5)

0.844551282051
0.873761655012
0.840399184149
0.853656759907
0.86166958042
0.854807692308


In [26]:
for i in range(5):
    print (i)
    user_similarity = train_user_based(split_matrices[i][0])
    np.savetxt('./epinions_subset/user_similarity_%d.txt'%i, user_similarity)

0
(1000, 3000)


KeyboardInterrupt: 

In [82]:
np.seterr(all='raise')
def nmae_user_user(test_matrix, train_matrix, similarity_matrix):
    indices = test_matrix.nonzero()
    y_true = []
    y_pred = []
    for i,j in zip(indices[0], indices[1]):
        y_true.append(test_matrix[i][j])
        user_sum = np.sum(similarity_matrix[i]) - 1
        if user_sum == 0:
            predicted_rating = np.mean(train_matrix[i,:][train_matrix[i,:].nonzero()])
        elif len(train_matrix[i,:][train_matrix[i,:].nonzero()]) == 0:
            predicted_rating = np.mean(train_matrix[:,j][train_matrix[:,j].nonzero()])
        else:
            numerator = 0
            for k in range(test_matrix.shape[0]):
                if k != i and train_matrix[k][j] != 0:
                    if len(train_matrix[k,:][train_matrix[k,:].nonzero()]) == 0:
                        print (train_matrix[k,:][train_matrix[k,:].nonzero()])
                    u_a_mean = np.mean(train_matrix[k,:][train_matrix[k,:].nonzero()])
                    numerator += similarity_matrix[i][k] * (train_matrix[k][j] - u_a_mean)
            predicted_rating = np.mean(train_matrix[i,:][train_matrix[i,:].nonzero()]) + numerator/user_sum
        y_pred.append(np.round(predicted_rating))
    return MAE(y_true, y_pred)

In [83]:
##prediction user user
count = 0
for i in range(5):
    similarity = np.loadtxt('./epinions_subset/user_similarity_%d.txt'%i)
    mae = nmae_user_user(split_matrices[i][1], split_matrices[i][0], similarity)
    count += mae
    print (mae)
print (count /5)

0.844478438228
0.875582750583
0.836611305361
0.853583916084
0.861451048951
0.854341491841


In [40]:
def nmae_trust(test_matrix, train_matrix, similarity_matrix, trust_matrix):
    similarity_matrix = 0.1*similarity_matrix + 0.9*trust_matrix
    indices = test_matrix.nonzero()
    y_true = []
    y_pred = []
    for i,j in zip(indices[0], indices[1]):
        y_true.append(test_matrix[i][j])
        user_sum = np.sum(similarity_matrix[i]) - 1
        if user_sum == 0:
            predicted_rating = np.mean(train_matrix[i,:][train_matrix[i,:].nonzero()])
        elif len(train_matrix[i,:][train_matrix[i,:].nonzero()]) == 0:
            predicted_rating = np.mean(train_matrix[:,j][train_matrix[:,j].nonzero()])
        else:
            numerator = 0
            for k in range(test_matrix.shape[0]):
                if k != i and train_matrix[k][j] != 0:
                    if len(train_matrix[k,:][train_matrix[k,:].nonzero()]) == 0:
                        print (train_matrix[k,:][train_matrix[k,:].nonzero()])
                    u_a_mean = np.mean(train_matrix[k,:][train_matrix[k,:].nonzero()])
                    numerator += similarity_matrix[i][k] * (train_matrix[k][j] - u_a_mean)
            predicted_rating = np.mean(train_matrix[i,:][train_matrix[i,:].nonzero()]) + numerator/user_sum
        y_pred.append(np.round(predicted_rating))
    return MAE(y_true, y_pred)/4

In [38]:
trustMat_E = None
trustMat_D = None
with h5py.File('trust_pairs.h5', 'r') as hf:
    trustMat_E = hf['E'][:]
    trustMat_D = hf['D'][:]

In [41]:
##prediction trust user
for i in range(5):
    similarity = np.loadtxt('./epinions_subset/user_similarity_%d.txt'%i)
    print (nmae_trust(split_matrices[i][1], split_matrices[i][0], similarity, trustMat_D))

0.211119609557
0.218895687646
0.20915282634
0.213395979021
0.215362762238


In [60]:
trust_embedding = np.loadtxt("./epinions_subset/epinions_embed_1", skiprows=1)
trust_embedding.sort(axis=0)
# padding = np.zeros((3000-trust_embedding.shape[0], trust_embedding.shape[1])) + 1
# trust_embedding = np.concatenate((trust_embedding,padding))[:,1:]
trust_embedding = trust_embedding[:,1:]
trust_embedding.shape

(3000, 128)

In [69]:
def train_embeddings(user_by_movie):
    similarity_matrix = np.zeros((user_by_movie.shape[0],user_by_movie.shape[0]))
    print (user_by_movie.shape)

    count = 0
    for i in range(user_by_movie.shape[0]):
        user1_movies = np.array(user_by_movie[i].T).flatten()
        mean_1 = np.mean(user1_movies)
#         mean_1 = np.average(user1_movies[user1_movies.nonzero()])
        start = time.clock()

        for j in range(user_by_movie.shape[0]):
            if i != j:
                user2_movies = np.array(user_by_movie[j].T).flatten()
#                 bitmap1 = user1_movies.nonzero()[0]
#                 bitmap2 = user2_movies.nonzero()[0]
#                 if bitmap2.shape[0] == 0:
#                     continue
#                 if bitmap1.shape[0] > bitmap2.shape[0]:
#                     bitmap2 = np.concatenate( (bitmap2, np.array(np.zeros( (1, bitmap1.shape[0] - bitmap2.shape[0]) ) ).flatten() ), axis = 0)
#                 else:
#                     bitmap1 = np.concatenate( (bitmap1, np.array(np.zeros( (1, bitmap2.shape[0] - bitmap1.shape[0]) ) ).flatten() ), axis = 0)
#                 bitmap = np.intersect1d(np.array(bitmap1).flatten(), np.array(bitmap2).flatten())
#                 # print bitmap
#                 common_ratings_user1 = np.array(user1_movies[bitmap.astype(int)]).flatten()
#                 common_ratings_user2 = np.array(user2_movies[bitmap.astype(int)]).flatten()

#                 if len(bitmap) > 0:
#                     mean_2 = np.average(user2_movies[user2_movies.nonzero()])
                mean_2 = np.mean(user2_movies)
                try:
                    user1_movies -= mean_1
                    user2_movies -= mean_2
                    sq_1 = np.array(np.square(user1_movies)).flatten()
                    sq_2 = np.array(np.square(user2_movies)).flatten()
                    s_score = np.sum( user1_movies * user2_movies )/ np.sqrt( np.sum(sq_1) * np.sum(sq_2) )
                    similarity_matrix[i][j] = s_score
                except Exception as e:
                    similarity_matrix[i][j] = 0
#                 else:
#                     similarity_matrix[i][j] = 0

        print ('Time Taken: ' + str(time.clock() - start))

        count += 1
        # if count % 10 == 0:
        print ('\t%d'%(count))
    return similarity_matrix
    
embedding_similarity = train_embeddings(trust_embedding)

(3000, 128)
Time Taken: 0.09999500000003536
	1
Time Taken: 0.09814600000004248
	2
Time Taken: 0.1359669999999369
	3
Time Taken: 0.1129120000000512
	4
Time Taken: 0.11760100000003604
	5
Time Taken: 0.1145050000000083
	6
Time Taken: 0.11616600000002109
	7
Time Taken: 0.1080019999999422
	8
Time Taken: 0.11035500000002685
	9
Time Taken: 0.1129320000000007
	10
Time Taken: 0.10696200000006684
	11
Time Taken: 0.11822800000004463
	12
Time Taken: 0.11906900000008136
	13
Time Taken: 0.11404799999991155
	14
Time Taken: 0.10419200000001183
	15
Time Taken: 0.13421099999993658
	16
Time Taken: 0.12740800000005947
	17
Time Taken: 0.12974799999994957
	18
Time Taken: 0.1250999999999749
	19
Time Taken: 0.10034399999995003
	20
Time Taken: 0.13966099999993276
	21
Time Taken: 0.1353639999999814
	22
Time Taken: 0.11084900000003017
	23
Time Taken: 0.10466899999994439
	24
Time Taken: 0.13255500000002485
	25
Time Taken: 0.13370799999995597
	26
Time Taken: 0.09911800000008952
	27
Time Taken: 0.10908000000006268


Time Taken: 0.1051830000000109
	227
Time Taken: 0.1400219999999308
	228
Time Taken: 0.1068759999999429
	229
Time Taken: 0.10855800000001636
	230
Time Taken: 0.11428499999999531
	231
Time Taken: 0.10311999999998989
	232
Time Taken: 0.10143699999991895
	233
Time Taken: 0.09477800000001935
	234
Time Taken: 0.11883100000000013
	235
Time Taken: 0.09811999999999443
	236
Time Taken: 0.11293699999998807
	237
Time Taken: 0.12044000000003052
	238
Time Taken: 0.09753000000000611
	239
Time Taken: 0.09770800000001145
	240
Time Taken: 0.13787500000000819
	241
Time Taken: 0.1057420000000775
	242
Time Taken: 0.09647500000005493
	243
Time Taken: 0.14458299999989777
	244
Time Taken: 0.10038199999996777
	245
Time Taken: 0.09481700000003457
	246
Time Taken: 0.12056600000005346
	247
Time Taken: 0.09470499999997628
	248
Time Taken: 0.09403399999996509
	249
Time Taken: 0.1340460000000121
	250
Time Taken: 0.11868300000003273
	251
Time Taken: 0.09650399999998172
	252
Time Taken: 0.11724800000001778
	253
Time T

Time Taken: 0.107262999999989
	449
Time Taken: 0.11113100000000031
	450
Time Taken: 0.09817299999997431
	451
Time Taken: 0.10025400000006357
	452
Time Taken: 0.13605600000005325
	453
Time Taken: 0.12724500000001626
	454
Time Taken: 0.14303900000004433
	455
Time Taken: 0.10157600000002276
	456
Time Taken: 0.12890300000003663
	457
Time Taken: 0.11734400000000278
	458
Time Taken: 0.12825599999996484
	459
Time Taken: 0.12400500000001102
	460
Time Taken: 0.11355400000002192
	461
Time Taken: 0.10415399999999408
	462
Time Taken: 0.13989200000003166
	463
Time Taken: 0.10728299999993851
	464
Time Taken: 0.09931999999992058
	465
Time Taken: 0.139803000000029
	466
Time Taken: 0.15613200000007055
	467
Time Taken: 0.13750900000002275
	468
Time Taken: 0.10998700000004646
	469
Time Taken: 0.09755699999993794
	470
Time Taken: 0.11807999999996355
	471
Time Taken: 0.11771999999996297
	472
Time Taken: 0.09523999999998978
	473
Time Taken: 0.10505499999999302
	474
Time Taken: 0.0994029999999384
	475
Time T

Time Taken: 0.11849800000004507
	672
Time Taken: 0.13860299999998915
	673
Time Taken: 0.12152100000002974
	674
Time Taken: 0.12127699999996366
	675
Time Taken: 0.11404100000004291
	676
Time Taken: 0.11237999999991644
	677
Time Taken: 0.10700300000007701
	678
Time Taken: 0.1032989999999927
	679
Time Taken: 0.1372639999999592
	680
Time Taken: 0.10088299999995343
	681
Time Taken: 0.09462599999994836
	682
Time Taken: 0.10348499999997784
	683
Time Taken: 0.09602300000005926
	684
Time Taken: 0.13584399999990637
	685
Time Taken: 0.10453500000005533
	686
Time Taken: 0.11375599999996666
	687
Time Taken: 0.11074500000006537
	688
Time Taken: 0.11088099999994938
	689
Time Taken: 0.10504800000001069
	690
Time Taken: 0.09662300000002233
	691
Time Taken: 0.1432879999999841
	692
Time Taken: 0.1129720000000134
	693
Time Taken: 0.09536799999989398
	694
Time Taken: 0.13946199999998043
	695
Time Taken: 0.12108999999998105
	696
Time Taken: 0.10720000000003438
	697
Time Taken: 0.13663599999995313
	698
Time 

Time Taken: 0.12483599999995931
	897
Time Taken: 0.11465400000008685
	898
Time Taken: 0.09769700000003922
	899
Time Taken: 0.10787900000002537
	900
Time Taken: 0.12679900000000544
	901
Time Taken: 0.09604699999999866
	902
Time Taken: 0.10967099999993479
	903
Time Taken: 0.13982500000008713
	904
Time Taken: 0.10155999999994947
	905
Time Taken: 0.09902299999998831
	906
Time Taken: 0.13683400000002166
	907
Time Taken: 0.11248399999999492
	908
Time Taken: 0.09690699999998742
	909
Time Taken: 0.12969799999996212
	910
Time Taken: 0.09487699999999677
	911
Time Taken: 0.09691700000007586
	912
Time Taken: 0.125974000000042
	913
Time Taken: 0.10165200000005825
	914
Time Taken: 0.1023169999999709
	915
Time Taken: 0.1126360000000659
	916
Time Taken: 0.10546900000008463
	917
Time Taken: 0.09377500000005057
	918
Time Taken: 0.10753199999999197
	919
Time Taken: 0.13755000000003292
	920
Time Taken: 0.10355800000002091
	921
Time Taken: 0.12657600000000002
	922
Time Taken: 0.1054229999999734
	923
Time T

...

Time Taken: 0.13804700000002867
	1986
Time Taken: 0.09918800000002648
	1987
Time Taken: 0.11686399999996411
	1988
Time Taken: 0.11261100000001534
	1989
Time Taken: 0.0946840000000293
	1990
Time Taken: 0.1102809999999863
	1991
Time Taken: 0.09702000000004318
	1992
Time Taken: 0.09734200000002602
	1993
Time Taken: 0.1362060000000156
	1994
Time Taken: 0.10181999999997515
	1995
Time Taken: 0.1359109999999646
	1996
Time Taken: 0.11982599999998911
	1997
Time Taken: 0.1386329999999134
	1998
Time Taken: 0.10813400000006368
	1999
Time Taken: 0.13720999999998185
	2000
Time Taken: 0.1175269999999955
	2001
Time Taken: 0.10750799999993887
	2002
Time Taken: 0.1335189999999784
	2003
Time Taken: 0.1102570000000469
	2004
Time Taken: 0.141695000000027
	2005
Time Taken: 0.11220699999989847
	2006
Time Taken: 0.10135300000001735
	2007
Time Taken: 0.10900800000001709
	2008
Time Taken: 0.09519000000000233
	2009
Time Taken: 0.1363690000000588
	2010
Time Taken: 0.10960799999998017
	2011
Time Taken: 0.097007999

Time Taken: 0.12231400000007397
	2204
Time Taken: 0.1150279999999384
	2205
Time Taken: 0.09576599999991231
	2206
Time Taken: 0.1335870000000341
	2207
Time Taken: 0.10629800000003797
	2208
Time Taken: 0.0950310000000627
	2209
Time Taken: 0.13314900000000307
	2210
Time Taken: 0.1001890000000003
	2211
Time Taken: 0.12087899999994534
	2212
Time Taken: 0.10807099999999537
	2213
Time Taken: 0.09491400000001704
	2214
Time Taken: 0.1407570000000078
	2215
Time Taken: 0.10400400000003174
	2216
Time Taken: 0.09445400000004156
	2217
Time Taken: 0.1340660000000753
	2218
Time Taken: 0.10216700000000856
	2219
Time Taken: 0.1015229999999292
	2220
Time Taken: 0.14447499999994307
	2221
Time Taken: 0.1002210000000332
	2222
Time Taken: 0.10290899999995418
	2223
Time Taken: 0.1300980000000891
	2224
Time Taken: 0.10667200000000321
	2225
Time Taken: 0.09892700000000332
	2226
Time Taken: 0.13770899999997255
	2227
Time Taken: 0.10499000000004344
	2228
Time Taken: 0.09681699999998727
	2229
Time Taken: 0.1305529

Time Taken: 0.11655600000005961
	2421
Time Taken: 0.11506500000007236
	2422
Time Taken: 0.15033499999992728
	2423
Time Taken: 0.13091099999996914
	2424
Time Taken: 0.1260450000000901
	2425
Time Taken: 0.11831100000006245
	2426
Time Taken: 0.12967800000001262
	2427
Time Taken: 0.11359500000003209
	2428
Time Taken: 0.11157000000002881
	2429
Time Taken: 0.15153700000007575
	2430
Time Taken: 0.12521000000003824
	2431
Time Taken: 0.13524900000004436
	2432
Time Taken: 0.11429600000008122
	2433
Time Taken: 0.12444899999991321
	2434
Time Taken: 0.127837999999997
	2435
Time Taken: 0.125678999999991
	2436
Time Taken: 0.1279540000000452
	2437
Time Taken: 0.13024099999995542
	2438
Time Taken: 0.1177660000000742
	2439
Time Taken: 0.1128320000000258
	2440
Time Taken: 0.1240379999999277
	2441
Time Taken: 0.12322600000004513
	2442
Time Taken: 0.11016600000004928
	2443
Time Taken: 0.11675999999999931
	2444
Time Taken: 0.12280099999998129
	2445
Time Taken: 0.12098700000001372
	2446
Time Taken: 0.1433240

Time Taken: 0.10436099999992621
	2639
Time Taken: 0.1068880000000263
	2640
Time Taken: 0.12066600000002836
	2641
Time Taken: 0.11278900000002068
	2642
Time Taken: 0.10190700000009656
	2643
Time Taken: 0.09975299999996423
	2644
Time Taken: 0.09369100000003527
	2645
Time Taken: 0.1372240000000602
	2646
Time Taken: 0.11452299999996285
	2647
Time Taken: 0.11244999999996708
	2648
Time Taken: 0.10242400000004181
	2649
Time Taken: 0.14054800000008072
	2650
Time Taken: 0.10885299999995368
	2651
Time Taken: 0.14176399999996647
	2652
Time Taken: 0.10294199999998455
	2653
Time Taken: 0.1023890000000165
	2654
Time Taken: 0.1369480000000749
	2655
Time Taken: 0.10588600000005499
	2656
Time Taken: 0.09762399999999616
	2657
Time Taken: 0.12032299999998486
	2658
Time Taken: 0.1331360000000359
	2659
Time Taken: 0.09436599999992268
	2660
Time Taken: 0.1239420000000564
	2661
Time Taken: 0.101270999999997
	2662
Time Taken: 0.13619599999992715
	2663
Time Taken: 0.10891500000002452
	2664
Time Taken: 0.102637

Time Taken: 0.12359000000003562
	2858
Time Taken: 0.12033500000006825
	2859
Time Taken: 0.12812700000006316
	2860
Time Taken: 0.12171499999999469
	2861
Time Taken: 0.11859400000003006
	2862
Time Taken: 0.13416300000005776
	2863
Time Taken: 0.1354110000000901
	2864
Time Taken: 0.1238700000000108
	2865
Time Taken: 0.14179400000000442
	2866
Time Taken: 0.10545300000001134
	2867
Time Taken: 0.09455100000002403
	2868
Time Taken: 0.10881800000004205
	2869
Time Taken: 0.09856600000000526
	2870
Time Taken: 0.10193100000003597
	2871
Time Taken: 0.1261899999999514
	2872
Time Taken: 0.12761299999999665
	2873
Time Taken: 0.12165199999992637
	2874
Time Taken: 0.10735799999997653
	2875
Time Taken: 0.09948399999996127
	2876
Time Taken: 0.12198300000000017
	2877
Time Taken: 0.13670300000001134
	2878
Time Taken: 0.1437349999999924
	2879
Time Taken: 0.11605699999995522
	2880
Time Taken: 0.0986840000000484
	2881
Time Taken: 0.11351400000000922
	2882
Time Taken: 0.13316899999995258
	2883
Time Taken: 0.111

In [71]:
## user embeddings
for i in range(5):
    print (nmae_user_user(split_matrices[i][1], split_matrices[i][0], embedding_similarity))

0.211119609557
0.219041375291
0.20902534965
0.213395979021
0.215362762238
